In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd "/content/drive/Shareddrives/IA avanzada/Reto_ArcaContinental/Products"

/content/drive/Shareddrives/IA avanzada/Reto_ArcaContinental/Products


In [3]:
import xml.etree.ElementTree as ET
import json

# Parsear el archivo XML
tree = ET.parse("sitemap.xml")
root = tree.getroot()

# Función para eliminar un prefijo de los nombres de los elementos
def remove_prefix(element, prefix):
    for elem in element.iter():
        if elem.tag.startswith(prefix):
            elem.tag = elem.tag[len(prefix):]

# Eliminar los prefijos de los elementos
prefix_to_remove_sitemaps = "{http://www.sitemaps.org/schemas/sitemap/0.9}"
prefix_to_remove_google_image = "{http://www.google.com/schemas/sitemap-image/1.1}"
prefix_to_remove_google_pagemap = "{http://www.google.com/schemas/sitemap-pagemap/1.0}"
remove_prefix(root, prefix_to_remove_sitemaps)
remove_prefix(root, prefix_to_remove_google_image)
remove_prefix(root, prefix_to_remove_google_pagemap)

# Función para convertir un elemento XML en un diccionario
def xml_to_dict(element):
    result = {}
    for child in element:
        if child:
            child_dict = xml_to_dict(child)
            if child.tag in result:
                if type(result[child.tag]) is list:
                    result[child.tag].append(child_dict)
                else:
                    result[child.tag] = [result[child.tag], child_dict]
            else:
                result[child.tag] = child_dict
        else:
            result[child.tag] = child.text
    return result

# Convertir el elemento raíz a un diccionario
data = xml_to_dict(root)

# Convertir el diccionario a formato JSON
json_data = json.dumps(data, indent=4)

# Guardar el JSON en un archivo
#with open("output.json", "w") as json_file:
 #   json_file.write(json_data)

print("JSON guardado en 'output.json'")


JSON guardado en 'output.json'


In [4]:
import pandas as pd
# Cargar el JSON desde el archivo
with open("output.json", "r") as json_file:
    data = json.load(json_file)

# Acceder a la lista de objetos "url"
url_list = data["url"]

items = []
images_url = []
images_title = []

for i in range(len(url_list)):
    if "image" in url_list[i]:
        if type(url_list[i]["image"]) == dict:
            images_url.append(url_list[i]["image"]["loc"])
            images_title.append(url_list[i]["image"]["title"])
        else:
            for j in range(len(url_list[i]["image"])):
                images_url.append(url_list[i]["image"][j]["loc"])
                images_title.append(url_list[i]["image"][j]["title"])

In [6]:
df_images = pd.DataFrame({'URL': images_url, 'Description': images_title})

In [8]:
csv_filename = "url_image_data.csv"  # Nombre del archivo CSV que deseas
df_images.to_csv(csv_filename, index=False)

In [ ]:
%cd "/content/drive/Shareddrives/IA avanzada/Reto_ArcaContinental/Products/Refrescos"

/content/drive/Shareddrives/IA avanzada/Reto_ArcaContinental/Products/Refrescos


In [ ]:
import json
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os
from urllib.parse import urlsplit

# Descargar imágenes y guardarlas con los nombres originales
downloaded_images = []  # Para almacenar información de las imágenes descargadas

for url, title in zip(images_url, images_title):
    response = requests.get(url)
    if response.status_code == 200:
        # Extraer la extensión del archivo usando urlsplit
        parsed_url = urlsplit(url)
        file_extension =  os.path.splitext(parsed_url.path)[1]

        # Generar un nombre de archivo único utilizando la extensión original
        filename = f"{os.path.basename(parsed_url.path)}"

        # Abrir la imagen usando PIL (Python Imaging Library)
        image = Image.open(BytesIO(response.content))

        # Convertir la imagen a modo RGB si está en modo RGBA
        if image.mode == 'RGBA':
            image = image.convert('RGB')

        # Guardar la imagen con el nombre original
        image.save(filename)

        # Almacenar información de la imagen en la lista downloaded_images
        downloaded_images.append({
            'ImageName': filename,
            'ImageTitle': title
        })

print("Imágenes descargadas y guardadas con nombres originales.")

# Crear un DataFrame a partir de la lista downloaded_images
image_df = pd.DataFrame(downloaded_images)

# Puedes usar image_df para trabajar con la información de las imágenes y sus títulos.


Imágenes descargadas y guardadas con nombres originales.


In [ ]:
csv_filename = "image_data.csv"  # Nombre del archivo CSV que deseas
image_df.to_csv(csv_filename, index=False)

print(f"DataFrame guardado como {csv_filename}")

DataFrame guardado como image_data.csv


In [ ]:
len(images_title)

610